In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [4]:
%pwd

'c:\\Users\\prana\\Downloads\\SourceCode Analysis\\research'

In [3]:
!mkdir test_repo

In [7]:
repo_path = "test_repo/"
repo = Repo.clone_from("https://github.com/PRANAVBALAJIRS/VisionRevive", to_path=repo_path)

In [8]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500))

In [9]:
documents = loader.load()

In [10]:
documents

[Document(page_content="from setuptools import setup, find_packages\n\nsetup(\n    name='ffa_net_project',\n    version='0.1',\n    packages=find_packages('src'),\n    package_dir={'': 'src'},\n    install_requires=[\n        'numpy',\n        'torch',\n        'scikit-learn',\n        'matplotlib',\n        'pandas',\n    ],\n    entry_points={\n        'console_scripts': [\n            'ffa_net=src.main:main',\n        ],\n    },\n)\n", metadata={'source': 'test_repo\\setup.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='import os, sys\nimport time, math\nimport argparse, random\nfrom math import exp\nimport numpy as np\n\nimport torch\nfrom torch import nn, optim\nimport torch.nn.functional as F\nimport torch.utils.data as data\nfrom torch.utils.data import DataLoader\nfrom torch.backends import cudnn\nfrom torch.autograd import Variable\n\nimport torchvision\nimport torchvision.transforms as tfs\nfrom torchvision.transforms import ToPILImage\nfrom torchvisio

In [11]:
len(documents)

15

In [12]:
document_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                                 chunk_size = 500,
                                                                 chunk_overlap = 20)

In [13]:
texts = document_splitter.split_documents(documents)

In [14]:
texts

[Document(page_content="from setuptools import setup, find_packages\n\nsetup(\n    name='ffa_net_project',\n    version='0.1',\n    packages=find_packages('src'),\n    package_dir={'': 'src'},\n    install_requires=[\n        'numpy',\n        'torch',\n        'scikit-learn',\n        'matplotlib',\n        'pandas',\n    ],\n    entry_points={\n        'console_scripts': [\n            'ffa_net=src.main:main',\n        ],\n    },\n)", metadata={'source': 'test_repo\\setup.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='import os, sys\nimport time, math\nimport argparse, random\nfrom math import exp\nimport numpy as np\n\nimport torch\nfrom torch import nn, optim\nimport torch.nn.functional as F\nimport torch.utils.data as data\nfrom torch.utils.data import DataLoader\nfrom torch.backends import cudnn\nfrom torch.autograd import Variable', metadata={'source': 'test_repo\\src\\config.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content="import 

In [15]:
len(texts)

54

In [16]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [17]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [18]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [19]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')

Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given


In [20]:
vectordb.persist()

In [21]:
llm = ChatOpenAI()

In [22]:
memory = ConversationSummaryMemory(llm = llm, memory_key = "chat_history", return_messages=True)

In [23]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"K":8}), memory=memory)

In [24]:
question = "What is PerLoss Class"

In [25]:
result = qa(question)

In [26]:
print(result['answer'])

The PerLoss class is a subclass of torch.nn.Module used for defining a perceptual loss function. It takes a VGG model as input and is used for calculating perceptual loss in neural network training.
